In [2]:
#importing essential modules
import pandas as pd
import numpy as np
import os,sys
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler
import tensorflow as tf

In [3]:
#importing local modules
sys.path.append(os.path.abspath(os.path.join('../scripts')))

from clean import Clean
from utils import vocab
from deep_learner import DeepLearn
from modeling import Modeler
from evaluator import CallbackEval

In [4]:
#mapping an object
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [5]:
AM_ALPHABET='ሀለሐመሠረሰቀበግዕዝተኀነአከወዐዘየደገጠጰጸፀፈፐቈኈጐኰፙፘፚauiāeəo'
EN_ALPHABET='abcdefghijklmnopqrstuvwxyz'

In [6]:
#create cleaner object to numerate every amharic alphabets
cleaner = Clean()
char_to_num,num_to_char=vocab(AM_ALPHABET)

2022-06-06 19:38:32,213:logger:Successfully initialized clean class


The vocabulary is: ['', 'ሀ', 'ለ', 'ሐ', 'መ', 'ሠ', 'ረ', 'ሰ', 'ቀ', 'በ', 'ግ', 'ዕ', 'ዝ', 'ተ', 'ኀ', 'ነ', 'አ', 'ከ', 'ወ', 'ዐ', 'ዘ', 'የ', 'ደ', 'ገ', 'ጠ', 'ጰ', 'ጸ', 'ፀ', 'ፈ', 'ፐ', 'ቈ', 'ኈ', 'ጐ', 'ኰ', 'ፙ', 'ፘ', 'ፚ', 'a', 'u', 'i', 'ā', 'e', 'ə', 'o'] (size =44)


# Deep Learning 

In [9]:
#read the csv of each dataset into their respective dataframes
swahili_df = pd.read_csv('../data/swahili.csv')
lang = pd.read_csv("../data/swahili.csv")
lang['type']='swahili'

amharic_df = pd.read_csv("../data/amharic.csv")
amharic_df['type']='amharic'
language_df = lang.append(amharic_df, ignore_index=True) #used to append two datasets and combine them in to a single dataset

In [10]:
#create an instance of modeler class
pre_model = Modeler()

In [11]:
#creating pre processed swahilli for deep learning
swahili_preprocessed = pre_model.preprocessing_learn(swahili_df,'key','text')